In [74]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entire‰ly on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
import time

## Group Assignment
### Team Number: 04
### Team Member Names: Arav Talati, Iris Hu, Nyra Rodrigues
### Team Strategy Chosen: Market Beat

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.


In [77]:
stock_csv = "Tickers_Example.csv"

# Read the stock symbols from CSV
stocks = pd.read_csv(stock_csv, header=None)
stocks.columns = ['Column1']

start_date = "2023-10-01"
end_date = "2024-09-30"

stock_list = []

# Loop through each ticker
for tckr in stocks["Column1"]:
    stock = yf.Ticker(tckr)

    # Check if the stock has 'currency' in the info
    if "currency" in stock.info.keys():
        stock_currency = stock.info["currency"]
        
        # Get the historical data for the ticker with monthly intervals
        stock_hist = stock.history(start=start_date, end=end_date, interval="1mo")
        time.sleep(0.3)

        # Calculate the average volume over the valid months
        if not stock_hist.empty:  # If there's data after dropping NaN volumes
            stock_volume = stock_hist['Volume'].dropna().mean()

            # Check if currency is USD or CAD and if the volume is above 100,000
            if (stock_currency == "USD" or stock_currency == "CAD") and stock_volume > 100000:
                stock_list.append(tckr)



In [78]:
start_date = "2024-09-01"
end_date = datetime.today()

MarketIndex1 = "^GSPTSE"  # TSX 60 #confirm ticker
MarketIndex2 = "^GSPC"  # S&P 500


market_hist1 = yf.Ticker(MarketIndex1).history(start=start_date, end=end_date)['Close']
market_hist2 = yf.Ticker(MarketIndex2).history(start=start_date, end=end_date)['Close']

market_hist1["Returns"] = market_hist1.pct_change() * 100
market_hist2["Returns"] = market_hist2.pct_change() * 100

market_avg_returns = (market_hist1["Returns"] + market_hist2["Returns"]) / 2

market_var = market_avg_returns.var()

# get_stock_beta returns the beta of the ticker inputted
def beta_val(stock_returns):
    df = pd.DataFrame({"Market Returns": market_avg_returns, "Stock Returns": stock_returns}).dropna()
    covariance = df.cov().iloc[0, 1]
    return covariance / market_var


#betas_list takes a list of tickers and outputs a list of all the betas of every ticker in the list
def betas_list(ticker_list):
    betas = {}
    for ticker in ticker_list:
        stock_hist = yf.Ticker(ticker).history(start=start_date, end=end_date)['Close']
        time.sleep(0.3)
        ticker_returns = stock_hist.pct_change()*100 
        betas[ticker] = beta_val(ticker_returns)
    return betas


all_betas = betas_list(stock_list)

In [79]:
beta_dict = betas_list(all_betas.keys())  #change based on taken
beta_df = pd.DataFrame.from_dict(beta_dict, orient='index')
beta_df.columns = ['Beta']
beta_df.index.name = 'Ticker'

In [80]:
#Sorts the dictionary from lowest beta to highest beta
sorted_beta_df = beta_df.sort_values('Beta').copy()

#Creates a dataframe with the 10 highest betas
#eligible_stocks = sorted_beta_df.loc[sorted_beta_df['Beta'] >= 1]

if (len(sorted_beta_df.loc[sorted_beta_df['Beta'] >= 1].index) < 12):
    eligible_stocks = sorted_beta_df.iloc[-15:]
elif ((len(sorted_beta_df.loc[sorted_beta_df['Beta'] >= 1].index) > 30)): 
    eligible_stocks = sorted_beta_df.iloc[-30:] 
else:
    eligible_stocks = sorted_beta_df.loc[sorted_beta_df['Beta'] >= 1]

In [81]:
def calculate_sharpe_ratio(ticker, risk_free_rate):
    stock_data = yf.Ticker(ticker).history(start=start_date, end=end_date)['Close']
    time.sleep(0.3)
    stock_returns = stock_data.pct_change()
    average_return = stock_returns.mean()
    std_deviation = stock_returns.std()
    sharpe_ratio = (average_return - risk_free_rate) / std_deviation
    return sharpe_ratio


sharpe_ratios = {}
for ticker in eligible_stocks.index:
    sharpe_ratios[ticker] = calculate_sharpe_ratio(ticker, 0)


for ticker, sharpe_ratio in sharpe_ratios.items():
    eligible_stocks.loc[ticker, "Sharpe Ratio"] = sharpe_ratio

eligible_stocks = eligible_stocks.sort_values('Sharpe Ratio', ascending=False)

if (len(eligible_stocks[eligible_stocks['Sharpe Ratio'] > 0]) < 12):
    eligible_stocks = eligible_stocks.iloc[:12]
else:
    lowSharpe = eligible_stocks[eligible_stocks['Sharpe Ratio'] <= 0].index
    eligible_stocks.drop(lowSharpe, inplace=True)

C:\Users\irish\AppData\Local\Temp\ipykernel_31208\219478237.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eligible_stocks.loc[ticker, "Sharpe Ratio"] = sharpe_ratio


In [82]:
for tckr in eligible_stocks.index:
    try:
        ticker = yf.Ticker(tckr)
        ticker_hist = ticker.history(start=start_date, end=end_date)
        time.sleep(0.3)
        ticker_hist['EMA Short'] = ticker_hist['Close'].ewm(span=2, adjust=True, min_periods=5).mean()
        ticker_hist['EMA Long'] = ticker_hist['Close'].ewm(span=50, adjust=True, min_periods=5).mean()
        if ticker_hist.empty:
            eligible_stocks.loc[tckr, 'Variance in EMA'] = -10000
        eligible_stocks.loc[tckr, 'Variance in EMA'] = ticker_hist['EMA Short'].iloc[len(ticker_hist)-1] - ticker_hist['EMA Long'].iloc[len(ticker_hist)-1]
    except:
        pass

eligible_stocks = eligible_stocks.sort_values('Variance in EMA', ascending=False)

if (len(eligible_stocks[eligible_stocks['Variance in EMA'] > -2]) < 12):
    eligible_stocks = eligible_stocks.iloc[:12]
elif (len(eligible_stocks[eligible_stocks['Variance in EMA'] > -2]) > 24):
    eligible_stocks = eligible_stocks.iloc[:24]
else:
    lowReturns = eligible_stocks[eligible_stocks['Variance in EMA'] <= -2].index
    eligible_stocks.drop(lowReturns, inplace=True)


eligible_stocks.dropna(axis=0, inplace=True)

In [83]:
sector_etfs = {
    "Technology": "XLK",
    "Healthcare": "XLV",
    "Financial Services": "XLF",
    "Consumer Cyclical": "XLY",
    "Energy": "XLE",
    "Utilities": "XLU",
    "Consumer Defensive": "XLP",
    "Industrials": "XLB",
    "Real Estate": "XLRE",
    "Communication Services": "XLC"
}

# Function to get P/E ratio for sector ETFs
def get_sector_pe_data(sector_etfs):
    sector_pe_data = {}
    for sector, etf in sector_etfs.items():
        
        # Fetch ETF data
        sector_etf = yf.Ticker(etf)
        info = sector_etf.info
        
        # Get the P/E ratio of the sector ETF
        pe_ratio = info.get('trailingPE', None)
        
        if pe_ratio:
            sector_pe_data[sector] = pe_ratio
    
    return sector_pe_data

# Get sector P/E data for each ETF
sector_pe_data = get_sector_pe_data(sector_etfs)

In [84]:
def get_stock_pe(tickers):
    stock_data = {}
    for ticker in tickers:
        
        stock = yf.Ticker(ticker)
        # Get stock info
        info = stock.info
        sector = info.get('sector', 'N/A')
        pe_ratio = info.get('trailingPE', None)  # trailing P/E ratio
        pb_ratio = info.get('priceToBook', None)
        
        # Add data to dictionary
        if (sector != 'N/A'): 
            if pe_ratio:
                stock_data[ticker] = {'sector': sector, 'PE': pe_ratio}
            else:
                stock_data[ticker] = {'sector': sector, 'PE': None}
        else:
            if pe_ratio:
                stock_data[ticker] = {'sector': 'N/A', 'PE': pe_ratio}
            else:
                stock_data[ticker] = {'sector': 'N/A', 'PE': None}

    return stock_data

stock_pe = get_stock_pe(eligible_stocks.index)

In [85]:
def compare_pe_to_sector(stock_data, sector_pe_data):
    # Print comparison of P/E ratios for each stock

    for tckr, data in stock_data.items():
        sector = data['sector']
        pe = data['PE']
        #pb = data['PB']
        
        if sector != 'N/A' and pe is not None:
            pe_sector_avg = sector_pe_data.get(sector, None)
            if pe_sector_avg is not None:
                eligible_stocks.loc[tckr, "comparisonPE"] = pe - pe_sector_avg
        else:
            eligible_stocks.loc[tckr, "comparisonPE"] = 0

compare_pe_to_sector(stock_pe, sector_pe_data)

for tckr in eligible_stocks.index:
    if eligible_stocks.loc[tckr, "comparisonPE"] == 0:
        eligible_stocks.loc[tckr, "comparisonPE"] = eligible_stocks["comparisonPE"].median()

eligible_stocks

,Beta,Sharpe Ratio,Variance in EMA,comparisonPE
Ticker,,,,
SHOP.TO,2.315268,0.234617,24.829635,55.634344
AXP,1.766465,0.154310,12.984568,-1.223573
AMZN,1.997064,0.120563,4.156779,8.820313
C,1.887320,0.115077,3.772301,-3.123817
PYPL,1.349022,0.160883,3.520474,-2.475824
BAC,1.535392,0.151914,3.161396,-5.934356
USB,1.550363,0.116997,2.728324,-6.955383
ACN,1.084695,0.072816,2.565507,-12.705385
CAT,1.931684,0.135296,1.788562,0.070747


In [86]:
eligible_stocks['Ranking'] = 0
    
length = len(eligible_stocks.index)
eligible_stocks = eligible_stocks.sort_values("Beta", ascending=False)

for i in range(length, 0, -1):
    eligible_stocks.loc[eligible_stocks.index[length-i], 'Ranking'] += i

eligible_stocks = eligible_stocks.sort_values("Sharpe Ratio", ascending=False)

for i in range(length, 0, -1):
    eligible_stocks.loc[eligible_stocks.index[length-i], 'Ranking'] += i

eligible_stocks = eligible_stocks.sort_values("Variance in EMA", ascending=False)

for i in range(length, 0, -1):
    eligible_stocks.loc[eligible_stocks.index[length-i], 'Ranking'] += i

eligible_stocks = eligible_stocks.sort_values("comparisonPE", ascending=False)

for i in range(length, 0, -1):
    eligible_stocks.loc[eligible_stocks.index[length-i], 'Ranking'] += i

eligible_stocks = eligible_stocks.sort_values("Ranking", ascending=False)

eligible_stocks

,Beta,Sharpe Ratio,Variance in EMA,comparisonPE,Ranking
Ticker,,,,,
SHOP.TO,2.315268,0.234617,24.829635,55.634344,48
AMZN,1.997064,0.120563,4.156779,8.820313,38
AXP,1.766465,0.154310,12.984568,-1.223573,37
CAT,1.931684,0.135296,1.788562,0.070747,31
PYPL,1.349022,0.160883,3.520474,-2.475824,30
C,1.887320,0.115077,3.772301,-3.123817,28
BAC,1.535392,0.151914,3.161396,-5.934356,26
BB.TO,2.117430,0.057162,-0.056707,-2.799821,24
USB,1.550363,0.116997,2.728324,-6.955383,22


In [87]:
n = len(eligible_stocks) # number of stocks
min_weight = 100 / (2 * n)  # Minimum weight percentage
max_weight = 15.0  # Maximum weight percentage
    
# Calculate initial weights
total_points = sum(eligible_stocks['Ranking'])
initial_weights = (eligible_stocks['Ranking']/total_points) * 100 
    
# Identify names below minimum threshold
below_min = pd.DataFrame()
below_min['Initial Weights'] = initial_weights[initial_weights < min_weight]
    
# Set minimum weights for those below threshold
below_min['Final Weights'] = min_weight

# Identify names above minimum threshold
above_min = pd.DataFrame()
above_min['Initial Weights'] = initial_weights[initial_weights >= min_weight]

# Calculate remaining weight to distribute
total_min_weight = len(below_min) * min_weight
remaining_weight = 100 - total_min_weight
total_above_min = sum(above_min['Initial Weights'])
    
# Redistribute remaining weight proportionally
above_min['Final Weights'] = (above_min['Initial Weights'] / total_above_min) * remaining_weight
final_weights = np.array(above_min['Final Weights'].values.tolist())
above_min['Final Weights'] = np.where(final_weights > max_weight, max_weight, final_weights).tolist()

# Check if we need to redistribute excess weight
total_allocated = sum(above_min['Final Weights']) + sum(below_min['Final Weights'])

if total_allocated < 100:
    excess = 100 - total_allocated
    non_max_weights = above_min['Final Weights'][above_min['Final Weights'] < max_weight]
    total_non_max = sum(non_max_weights)
    
    additional = (non_max_weights/total_non_max) * excess
    above_min['Final Weights'] += additional

# Combine below_min and above_min dataframes
combined_weights = pd.concat([above_min, below_min])

# Ensure sum is exactly 100
remaining = 100.0
sorted_items = pd.DataFrame()
sorted_items['Final Weights'] = combined_weights['Final Weights'].sort_values(ascending=False)

for i in range(len(sorted_items)-1):
    remaining -= sorted_items.iloc[i+1, 0]
    
# Assign the remaining weight to the first item to ensure sum is exactly 100
sorted_items.iloc[0, 0] = remaining

eligible_stocks['Weights'] = sorted_items['Final Weights']
eligible_stocks

,Beta,Sharpe Ratio,Variance in EMA,comparisonPE,Ranking,Weights
Ticker,,,,,,
SHOP.TO,2.315268,0.234617,24.829635,55.634344,48,14.788732
AMZN,1.997064,0.120563,4.156779,8.820313,38,11.707746
AXP,1.766465,0.154310,12.984568,-1.223573,37,11.399648
CAT,1.931684,0.135296,1.788562,0.070747,31,9.551056
PYPL,1.349022,0.160883,3.520474,-2.475824,30,9.242958
C,1.887320,0.115077,3.772301,-3.123817,28,8.626761
BAC,1.535392,0.151914,3.161396,-5.934356,26,8.010563
BB.TO,2.117430,0.057162,-0.056707,-2.799821,24,7.394366
USB,1.550363,0.116997,2.728324,-6.955383,22,6.778169


In [88]:
investment_money = 1000000

# convert dataframe column Weights to dictionary
weights = eligible_stocks['Weights'].to_dict()

Portfolio_Final = pd.DataFrame(eligible_stocks.index)

Portfolio_Final.index = Portfolio_Final.index + 1

total_value = 0

exchange_rate = yf.Ticker("USDCAD=x")

conversion_rate = exchange_rate.info["previousClose"]

for i in range(1, len(Portfolio_Final.index)+1):

    tckr = eligible_stocks.index[i-1]

    ticker = yf.Ticker(Portfolio_Final.loc[i, "Ticker"])

    Close = ticker.info["previousClose"]

    Portfolio_Final.loc[i, "Price"] = Close

    Currency = ticker.info["currency"]

    Portfolio_Final.loc[i, "Currency"] = Currency

    if (Portfolio_Final.loc[i, "Currency"] == "USD"):
        Close = Close * conversion_rate

    pot_fees = ((weights[tckr]/100)*investment_money)/Close*0.001

    if pot_fees > 3.95:
        money = ((weights[tckr]/100)*investment_money) - 3.95
    else:
        money = ((weights[tckr]/100)*investment_money) - pot_fees

    Shares = money/Close

    Portfolio_Final.loc[i, "Shares"] = Shares

    Portfolio_Final.loc[i, "Value"] = Shares*Close

    Portfolio_Final.loc[i, "Weight"] = (Portfolio_Final.loc[i, "Value"]/investment_money) * 100

    if pot_fees > 3.95:
        total_value +=  Portfolio_Final.loc[i, "Value"] + 3.95
    else:
        total_value +=  Portfolio_Final.loc[i, "Value"] + (Portfolio_Final.loc[i, "Shares"]*0.001)


print ("The total value of the Portfolio is: $", round(total_value, 2))
total_weight = sum(Portfolio_Final["Weight"])

print("The total weight of each of the stocks in the portfolio add up to: ", round(total_weight, 2), "%", )

Portfolio_Final

The total value of the Portfolio is: $ 1000000.0
The total weight of each of the stocks in the portfolio add up to:  100.0 %


,Ticker,Price,Currency,Shares,Value,Weight
1,SHOP.TO,145.34,CAD,1017.519653,147886.306417,14.788631
2,AMZN,202.88,USD,413.046391,117077.051741,11.707705
3,AXP,287.71,USD,283.597038,113996.195275,11.399620
4,CAT,381.50,USD,179.193536,95510.384186,9.551038
5,PYPL,84.74,USD,780.701709,92428.796756,9.242880
6,C,68.28,USD,904.307160,86266.701317,8.626670
7,BAC,46.06,USD,1244.797389,80104.388986,8.010439
8,BB.TO,3.24,CAD,22820.898757,73939.711972,7.393971
9,USB,50.74,USD,956.141338,67780.733986,6.778073
10,ACN,357.07,USD,83.521986,41666.583145,4.166658


Note that the weight shown in the table are not greater than 100/2n% (where n is the number of stocks in the portfolio), as we are comparing the value against the full $1000000 of investment money, which doesn't take into account the fees that impact the investment weight value.

In [90]:
Stocks_Final = Portfolio_Final[['Ticker','Shares']]
Stocks_Final.to_csv('Stocks_Group_04.csv')

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.